In [4]:
import pandas as pd
import pymssql
from sqlalchemy import create_engine
from datetime import datetime
from sklearn.preprocessing import QuantileTransformer
from joblib import load

qt = QuantileTransformer(output_distribution="normal")
pipe = load("files/linear_model_2023-01-20.joblib")

today = datetime.today().strftime("%Y-%m-%d")

In [5]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [6]:
df = pd.read_sql(
    """
    select ticker, bm, mom12m, roeq, mve, famaindustry
    from today
    """, 
    conn
)
df = df.dropna()
df = df.set_index("ticker")

In [7]:
df = df.sort_values(by="mve")
df = df.iloc[:-500]

In [8]:
features = ["bm", "mom12m", "roeq"]

transformed_features = qt.fit_transform (df[features])
transformed_features = pd.DataFrame(transformed_features, columns=features, index=df.index)

df["predict"] = pipe.predict(transformed_features)

for f in features:
    df["t" + f] = transformed_features[f]
    
df.to_csv(f"files/predictions_{today}.csv")